# Functions, Tools & Agents

En este notebook veremos cómo crear funciones, herramientas y agentes


## Configuración de la API de OpenAI
Este código importa las librerías necesarias para trabajar con la API de OpenAI, carga las variables de entorno desde un archivo .env y configura la clave de la API de OpenAI para ser utilizada en las peticiones a la API.


In [1]:
!pip install langchain openai langchain_experimental langchain_openai tiktoken wikipedia langchainhub

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=4e6d57a57d0cb8f79b37d94a532588147b2552d7cbeb743fe610e5d40fae8c08
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.59
    Uninstalling langchain-core-0.3.59:
      Successfully uninstalled langchain-core-0.3.59


In [2]:
import os
import getpass

api_key = getpass.getpass("Enter your OpenAI API Key:")

Enter your OpenAI API Key:··········


## Function Calling

https://dev.pokemontcg.io/dashboard

In [3]:
import requests

def get_most_expensive_pokemon_card(card_name):
    BASE_URL = "https://api.pokemontcg.io/v2/cards"
    params = {"q": f"name:{card_name}"}
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        cards = response.json().get("data", [])
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    if not cards:
        return "Pokémon card not found"

    most_expensive_card = max(cards, key=lambda card: card.get("tcgplayer", {}).get("prices", {}).get("holofoil", {}).get("market", 0) or 0)
    highest_price = most_expensive_card.get("tcgplayer", {}).get("prices", {}).get("holofoil", {}).get("market", "N/A")

    return f"Most Expensive {card_name} Card: {most_expensive_card['name']} - Market Price: ${highest_price}"

examples = ["Charizard", "Pikachu", "Blastoise", "Venusaur", "Mewtwo"]
for card in examples:
    print(get_most_expensive_pokemon_card(card))


Most Expensive Charizard Card: Charizard ★ δ - Market Price: $993.99
Most Expensive Pikachu Card: Pikachu ★ - Market Price: $2066.66
Most Expensive Blastoise Card: Blastoise - Market Price: $359.99
Most Expensive Venusaur Card: Venusaur ex - Market Price: $281.99
Most Expensive Mewtwo Card: Mewtwo ★ - Market Price: $2001.98


Definimos nuestra API con la estructura OpenAPI


In [4]:
functions = [
    {
      "name": "get_most_expensive_pokemon_card",
      "description": "Find the most expensive Pokémon card based on the Pokémon's name.",
      "parameters": {
        "type": "object",
        "properties": {
          "pokemon_name": {
            "type": "string",
            "description": "The name of the Pokémon to search for. Example values: 'Charizard', 'Pikachu', 'Blastoise', 'Venusaur', 'Mewtwo'."
          }
        },
        "required": ["pokemon_name"]
      }
    }
]

Definimos la función de completado con funciones como parámetro de entrada


In [5]:
from openai import OpenAI

client = OpenAI(api_key = api_key)

def get_completion_from_functions(messages, model="gpt-4.1-mini", temperature=0, functions = None):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        functions=functions
    )
    return response

Ahora pediremos al modelo cuál es el piso más caro de Encamp, a ver si es capaz de decidir utilizar la función que hemos creado.


In [14]:
messages = [
    {
        "role": "user",
        "content": "Cual es la carta más cara de Pikachu?"
    }
]

In [15]:
response = get_completion_from_functions(messages=messages, functions=functions)

In [16]:
print(response)

ChatCompletion(id='chatcmpl-Ba4tiVVjudreWwbXdbaxoKggWkE8s', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"pokemon_name":"Pikachu"}', name='get_most_expensive_pokemon_card'), tool_calls=None))], created=1747938334, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_71b9d4b387', usage=CompletionUsage(completion_tokens=23, prompt_tokens=105, total_tokens=128, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [18]:
response_message = response.choices[0].message
print(response_message.content)
print(response_message.function_call)
print(response_message.function_call.arguments)

None
FunctionCall(arguments='{"pokemon_name":"Pikachu"}', name='get_most_expensive_pokemon_card')
{"pokemon_name":"Pikachu"}


In [19]:
import json

args = json.loads(response_message.function_call.arguments)
print(args)

{'pokemon_name': 'Pikachu'}


Llamamos a la función utilizando los args que ha sido capaz de extraer de la frase del usuario


In [20]:
get_most_expensive_pokemon_card(args["pokemon_name"])

'Most Expensive Pikachu Card: Pikachu ★ - Market Price: $2066.66'

Ahora intentaremos con una pregunta que no tiene nada que ver, a ver si infiere no tener que utilizar la función


In [21]:
messages = [
    {
        "role": "user",
        "content": "Quien es el tio mas guapo de Andorra?"
    }
]

In [22]:
response = get_completion_from_functions(messages=messages, functions=functions)

In [23]:
response_message = response.choices[0].message
print(response_message.content)
print(response_message.function_call)

No tengo información específica sobre quién es considerado el "tío más guapo" de Andorra, ya que la belleza es subjetiva y depende de la opinión personal de cada individuo. ¿Te gustaría que te ayude con alguna otra consulta?
None


## Tools

Ahora utilizaremos Langchain para definir alguna herramienta para que posteriormente un LLM Agent pueda decidir usar la herramienta.


In [24]:
from typing import Optional
from langchain.agents import tool
from langchain.pydantic_v1 import BaseModel, Field

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [25]:
from typing import Optional
from pydantic import BaseModel, Field
import requests

class PokemonSearchInput(BaseModel):
    pokemon_name: str = Field(description="The name of the Pokémon to search for. Example values: 'Charizard', 'Pikachu', 'Blastoise', 'Venusaur', 'Mewtwo'.")

@tool('get-most-expensive-pokemon-card', args_schema=PokemonSearchInput, return_direct=True)
def get_most_expensive_pokemon_card(pokemon_name: str) -> str:
    """Find the most expensive Pokémon card based on the Pokémon's name."""

    BASE_URL = "https://api.pokemontcg.io/v2/cards"
    params = {"q": f"name:{pokemon_name}"}
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        cards = response.json().get("data", [])
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    if not cards:
        return "Pokémon card not found"

    most_expensive_card = max(cards, key=lambda card: card.get("tcgplayer", {}).get("prices", {}).get("holofoil", {}).get("market", 0) or 0)
    highest_price = most_expensive_card.get("tcgplayer", {}).get("prices", {}).get("holofoil", {}).get("market", "N/A")

    return f"Most Expensive {pokemon_name} Card: {most_expensive_card['name']} - Market Price: ${highest_price}"

In [26]:
get_most_expensive_pokemon_card.name

'get-most-expensive-pokemon-card'

In [27]:
get_most_expensive_pokemon_card.description

"Find the most expensive Pokémon card based on the Pokémon's name."

In [28]:
get_most_expensive_pokemon_card.args

{'pokemon_name': {'description': "The name of the Pokémon to search for. Example values: 'Charizard', 'Pikachu', 'Blastoise', 'Venusaur', 'Mewtwo'.",
  'title': 'Pokemon Name',
  'type': 'string'}}

Ahora vamos a crear la nueva herramienta


In [29]:
from langchain.tools.render import format_tool_to_openai_function

In [30]:
format_tool_to_openai_function(get_most_expensive_pokemon_card)

<ipython-input-30-75c00a1fc3f2>:1: LangChainDeprecationWarning: The function `_format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  format_tool_to_openai_function(get_most_expensive_pokemon_card)


{'name': 'get-most-expensive-pokemon-card',
 'description': "Find the most expensive Pokémon card based on the Pokémon's name.",
 'parameters': {'properties': {'pokemon_name': {'description': "The name of the Pokémon to search for. Example values: 'Charizard', 'Pikachu', 'Blastoise', 'Venusaur', 'Mewtwo'.",
    'type': 'string'}},
  'required': ['pokemon_name'],
  'type': 'object'}}

In [33]:
get_most_expensive_pokemon_card({"pokemon_name": 'pikachu'})

'Most Expensive pikachu Card: Pikachu ★ - Market Price: $2066.66'

Ahora crearemos otra herramienta, una que busque artículos en Wikipedia


In [34]:
import wikipedia

@tool('search-wikipedia')
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries. Execute this tool every time we need to explain information about a pokemon"""

    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [35]:
search_wikipedia.name

'search-wikipedia'

In [36]:
search_wikipedia.description

'Run Wikipedia search and get page summaries. Execute this tool every time we need to explain information about a pokemon'

In [37]:
format_tool_to_openai_function(search_wikipedia)

{'name': 'search-wikipedia',
 'description': 'Run Wikipedia search and get page summaries. Execute this tool every time we need to explain information about a pokemon',
 'parameters': {'properties': {'query': {'type': 'string'}},
  'required': ['query'],
  'type': 'object'}}

In [38]:
search_wikipedia({"query": "Pikachu"})

'Page: Pikachu\nSummary: Pikachu ( ; Japanese: ピカチュウ, Hepburn: Pikachū) is a Pokémon species in Nintendo and Game Freak\'s Pokémon media franchise, and the franchise\'s mascot. First introduced in the video games Pokémon Red and Blue, it was created by Atsuko Nishida at the request of lead designer Ken Sugimori, with the design finalized by Sugimori. Since Pikachu\'s debut, it has appeared in multiple games including Pokémon Go and the Pokémon Trading Card Game, as well as various merchandise. While Pikachu has been primarily voiced in media by Ikue Ōtani, other actors have also voiced the character including Kate Bristol, Ryan Reynolds, Kaiji Tang, Hidetoshi Nishijima, Tōru Ōkawa, and Koichi Yamadera.\nClassified as an Electric-type Pokémon, Pikachu is a large yellow mouse with a lightning bolt-shaped tail, and red sacs on its cheek which can generate large amounts of electricity. Originally designed to be the first part of a three-stage evolution line in Red and Blue, Pikachu evolves

## Tool Routing

Con las dos herramientas que hemos creado, crearemos un enrutador para que la LLM pueda decidir si usa o no la herramienta a partir de una consulta.


In [39]:
from langchain_openai import ChatOpenAI

In [49]:
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_most_expensive_pokemon_card
    ]
]
model = ChatOpenAI(temperature=0, api_key=api_key, model="gpt-4.1-mini").bind(functions=functions)

In [42]:
model.invoke("Que me puedes explicar de Pikachu")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Pikachu"}', 'name': 'search-wikipedia'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 140, 'total_tokens': 157, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Ba51apL3pGrv57qviMtBDph0ZJNQU', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--f1d76766-411c-483f-a5ae-d0c79eff1a32-0', usage_metadata={'input_tokens': 140, 'output_tokens': 17, 'total_tokens': 157, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [43]:
model.invoke("Cual es la carta más cara de Charizard?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"pokemon_name":"Charizard"}', 'name': 'get-most-expensive-pokemon-card'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 144, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Ba51p5A6rZQHGmy0EZDxh9ojGjckX', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--0615bb46-97f3-447d-8df9-4d1ec0dd6fc2-0', usage_metadata={'input_tokens': 144, 'output_tokens': 21, 'total_tokens': 165, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [44]:
model.invoke("Explicame cosas sobre Venusaur y dime cual es la carta más cara?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Venusaur"}', 'name': 'search-wikipedia'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 149, 'total_tokens': 166, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Ba529zt5NdKe4M1BKAwvDYogK0xEm', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--5f510e16-08ed-42ee-9938-eb1ab155c08f-0', usage_metadata={'input_tokens': 149, 'output_tokens': 17, 'total_tokens': 166, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Si usamos Lang**Chain** debemos usar CHAINS


In [46]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant who always answer in Spanish Pirate"),
    ("user", "{input}"),
])

chain = prompt | model

In [47]:
chain.invoke({"input": "Cual es el Mewto más caro?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"pokemon_name":"Mewtwo"}', 'name': 'get-most-expensive-pokemon-card'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 155, 'total_tokens': 177, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Ba53c3Ral1yMbGGZ51rAG7MN7kQqx', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--4f739b30-c851-4b15-9ef5-daee8a0f3c07-0', usage_metadata={'input_tokens': 155, 'output_tokens': 22, 'total_tokens': 177, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [50]:
chain.invoke({"input": "Que me puedes explicar de Charizard?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Charizard"}', 'name': 'search-wikipedia'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 153, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Ba54etotJgQfpr4vAHIDHsf4BS48s', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--88c6e856-de47-401f-b136-e8916d8c6b7f-0', usage_metadata={'input_tokens': 153, 'output_tokens': 16, 'total_tokens': 169, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Y el resultado?

In [51]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [52]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [53]:
result = chain.invoke({"input": "Explicame alguna cosa de Mewto"})

In [54]:
type(result)

langchain_core.agents.AgentActionMessageLog

In [55]:
result.tool

'search-wikipedia'

In [56]:
result.tool_input

{'query': 'Mewtwo'}

In [57]:
search_wikipedia(result.tool_input)

'Page: Mewtwo\nSummary: Mewtwo ( ; Japanese: ミュウツー, Hepburn: Myūtsū) is a Pokémon species in Nintendo and Game Freak\'s Pokémon media franchise. It was first introduced in the video games Pokémon Red and Blue, and later appeared in subsequent sequels and spin-off titles, such as Pokkén Tournament  and Detective Pikachu. In the video games, the player can fight and capture Mewtwo in order to subsequently pit it against other Pokémon. The player can first learn of Mewtwo late in Pokémon Red and Blue by reading research documents left in a ruined laboratory on Cinnabar Island where Mewtwo has escaped. Mewtwo is regarded as one of the series\' strongest Pokémon, often referred to as "the world\'s strongest Pokémon" in various media, and was the strongest in the original games in terms of base statistic distribution. It is known as the "Genetic  Pokémon" and is a Legendary Pokémon, a special group of Pokémon that are very rare and usually very powerful.  Mewtwo has also appeared in various 

Solo es necesario que enroutemos las respuestas


In [58]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search-wikipedia": search_wikipedia,
            "get-most-expensive-pokemon-card": get_most_expensive_pokemon_card,
        }
        return tools[result.tool].run(result.tool_input)

In [59]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [62]:
result = chain.invoke({"input": "Cual es el Mewto más caro??"})
result

'Most Expensive Mewtwo Card: Mewtwo ★ - Market Price: $2001.98'

In [63]:
result = chain.invoke({"input": "Que me puedes explicar de Leo Messi?"})
result

'Page: Lionel Messi\nSummary: Lionel Andrés "Leo" Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ; born 24 June 1987) is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players of all time, Messi set numerous records for individual accolades won throughout his professional footballing career such as eight Ballon d\'Or awards and eight times being named the world\'s best player by FIFA. He is the most decorated player in the history of professional football having won 45 team trophies, including twelve Big Five league titles, four UEFA Champions Leagues, two Copa Américas, and one FIFA World Cup. Messi holds the records for most European Golden Shoes (6), most goals in a calendar year (91), most goals for a single club (672, with Barcelona), most goals (474), hat-tricks (36) and assists (192) in La Liga, most assists (18) and goal contribu

In [64]:
result = chain.invoke({"input": "Explicame cosas sobre Mewto"})
result

'Page: Mewtwo\nSummary: Mewtwo ( ; Japanese: ミュウツー, Hepburn: Myūtsū) is a Pokémon species in Nintendo and Game Freak\'s Pokémon media franchise. It was first introduced in the video games Pokémon Red and Blue, and later appeared in subsequent sequels and spin-off titles, such as Pokkén Tournament  and Detective Pikachu. In the video games, the player can fight and capture Mewtwo in order to subsequently pit it against other Pokémon. The player can first learn of Mewtwo late in Pokémon Red and Blue by reading research documents left in a ruined laboratory on Cinnabar Island where Mewtwo has escaped. Mewtwo is regarded as one of the series\' strongest Pokémon, often referred to as "the world\'s strongest Pokémon" in various media, and was the strongest in the original games in terms of base statistic distribution. It is known as the "Genetic  Pokémon" and is a Legendary Pokémon, a special group of Pokémon that are very rare and usually very powerful.  Mewtwo has also appeared in various 

## Agents

Una vez tenemos las herramientas creadas, ¡solo hace falta añadirlas al contexto de un agente para que las pueda usar!


In [65]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

tools = [get_most_expensive_pokemon_card, search_wikipedia]

In [67]:
functions = [format_tool_to_openai_function(f) for f in tools]

model = ChatOpenAI(temperature=0, api_key=api_key, model="gpt-4.1-mini").bind(functions=functions)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant who always answer in Spanish Pirate"),
    ("user", "{input}"),
])

chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [68]:
result = chain.invoke({"input": "Cual es el Charizard más caro"})

In [69]:
result.tool

'get-most-expensive-pokemon-card'

In [70]:
result.tool_input

{'pokemon_name': 'Charizard'}

In [71]:
from langchain.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant who always answer in Spanish Pirate"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])


In [72]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [73]:
result1 = chain.invoke({
    "input": "Cual es el Pikachu más caro?",
    "agent_scratchpad": []
})

In [74]:
result1.tool

'get-most-expensive-pokemon-card'

In [75]:
pikachu = get_most_expensive_pokemon_card(result1.tool_input)
pikachu

'Most Expensive Pikachu Card: Pikachu ★ - Market Price: $2066.66'

In [76]:
result1.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"pokemon_name":"Pikachu"}', 'name': 'get-most-expensive-pokemon-card'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 150, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_79b79be41f', 'id': 'chatcmpl-Ba5ArIPdnvDMuTEVAXUIwHlb3Wsfa', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--2e2db918-195b-4f5d-ad66-1c5c7fa4ccc4-0', usage_metadata={'input_tokens': 150, 'output_tokens': 22, 'total_tokens': 172, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [77]:
from langchain.agents.format_scratchpad import format_to_openai_functions
format_to_openai_functions([(result1, pikachu), ])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"pokemon_name":"Pikachu"}', 'name': 'get-most-expensive-pokemon-card'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 150, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_79b79be41f', 'id': 'chatcmpl-Ba5ArIPdnvDMuTEVAXUIwHlb3Wsfa', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--2e2db918-195b-4f5d-ad66-1c5c7fa4ccc4-0', usage_metadata={'input_tokens': 150, 'output_tokens': 22, 'total_tokens': 172, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 FunctionMessage(content='Most Expensive Pikachu Card: Pikachu ★ - Market Price

Crearemos una cadena **agente** y que nos enseñe todos los pasos intermedios


In [78]:
from langchain.schema.runnable import RunnablePassthrough

agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

In [79]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [80]:
agent_executor.invoke({"input": "Que sabes de Pikachu?"})



> Entering new AgentExecutor chain...

Invoking: `search-wikipedia` with `{'query': 'Pikachu'}`


Page: Pikachu
Summary: Pikachu ( ; Japanese: ピカチュウ, Hepburn: Pikachū) is a Pokémon species in Nintendo and Game Freak's Pokémon media franchise, and the franchise's mascot. First introduced in the video games Pokémon Red and Blue, it was created by Atsuko Nishida at the request of lead designer Ken Sugimori, with the design finalized by Sugimori. Since Pikachu's debut, it has appeared in multiple games including Pokémon Go and the Pokémon Trading Card Game, as well as various merchandise. While Pikachu has been primarily voiced in media by Ikue Ōtani, other actors have also voiced the character including Kate Bristol, Ryan Reynolds, Kaiji Tang, Hidetoshi Nishijima, Tōru Ōkawa, and Koichi Yamadera.
Classified as an Electric-type Pokémon, Pikachu is a large yellow mouse with a lightning bolt-shaped tail, and red sacs on its cheek which can generate large amounts of electricity. Originally 

{'input': 'Que sabes de Pikachu?',
 'output': '¡Arrr, camarada! Pikachu es un Pokémon muy famoso, un ratoncito amarillo con cola en forma de rayo y mejillas rojas que pueden generar electricidad. Es el Pokémon eléctrico por excelencia y es la mascota del franquicia Pokémon. Fue creado por Atsuko Nishida y su diseño final fue hecho por Ken Sugimori. Pikachu apareció por primera vez en los juegos Pokémon Red y Blue, y desde entonces ha estado en muchos juegos, cartas y mercancías.\n\nEste pequeño eléctrico evoluciona a Raichu cuando se le da una piedra trueno, y tiene una pre-evolución llamada Pichu. Pikachu es muy popular, especialmente por ser el compañero del protagonista Ash Ketchum en la serie animada Pokémon.\n\nAdemás, Pikachu tuvo su propia película en acción real llamada "Detective Pikachu" en 2019, donde fue interpretado por la voz de Ryan Reynolds. ¡Un verdadero ícono de la cultura pop japonesa y del mundo Pokémon, arrr! ¿Quieres saber algo más sobre este electrizante Pokémon?

Añadiremos una plantilla de prompt con un prompt de sistema


In [81]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant who always answer in Spanish Pirate"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [82]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

¡Y le podemos añadir memoria a la cadena!


In [83]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

<ipython-input-83-4554adb1bf85>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")


In [88]:
agent_executor = AgentExecutor(agent=agent_chain,
                               tools=tools,
                               verbose=True,
                               memory=memory,
                               max_iterations=4)

In [89]:
agent_executor.invoke({"input": "Hola! mi nombre es Eric!"})



> Entering new AgentExecutor chain...
¡Ahoy de nuevo, Eric! ¡Listo para zarpar en otra aventura pirata! ¿Qué tesoro buscas hoy en estas aguas digitales?

> Finished chain.


{'input': 'Hola! mi nombre es Eric!',
 'chat_history': [HumanMessage(content='Hola! mi nombre es Eric!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='¡Ahoy, Eric! ¡Bienvenido a bordo, marinero! ¿En qué puedo ayudarte hoy en esta travesía pirata?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Que sabes sobre Pikachu?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='¡Arrr, Eric! Pikachu es un Pokémon muy famoso, un ratoncito amarillo con una cola en forma de rayo y mejillas rojas que pueden generar electricidad. Es el Pokémon eléctrico por excelencia y es la mascota del mundo Pokémon. Fue creado para los juegos Pokémon Rojo y Azul y desde entonces ha aparecido en muchos juegos, cartas y hasta en la serie animada como compañero del valiente Ash Ketchum.\n\nPikachu puede evolucionar a Raichu usando una piedra trueno, y tiene un pre-evolución llamado Pichu. Es muy querido por su diseño adorable y es un ícono no solo de Pokémon,

In [90]:
agent_executor.invoke({"input": "Que sabes sobre Pikachu?"})



> Entering new AgentExecutor chain...

Invoking: `search-wikipedia` with `{'query': 'Pikachu'}`


Page: Pikachu
Summary: Pikachu ( ; Japanese: ピカチュウ, Hepburn: Pikachū) is a Pokémon species in Nintendo and Game Freak's Pokémon media franchise, and the franchise's mascot. First introduced in the video games Pokémon Red and Blue, it was created by Atsuko Nishida at the request of lead designer Ken Sugimori, with the design finalized by Sugimori. Since Pikachu's debut, it has appeared in multiple games including Pokémon Go and the Pokémon Trading Card Game, as well as various merchandise. While Pikachu has been primarily voiced in media by Ikue Ōtani, other actors have also voiced the character including Kate Bristol, Ryan Reynolds, Kaiji Tang, Hidetoshi Nishijima, Tōru Ōkawa, and Koichi Yamadera.
Classified as an Electric-type Pokémon, Pikachu is a large yellow mouse with a lightning bolt-shaped tail, and red sacs on its cheek which can generate large amounts of electricity. Originally 

{'input': 'Que sabes sobre Pikachu?',
 'chat_history': [HumanMessage(content='Hola! mi nombre es Eric!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='¡Ahoy, Eric! ¡Bienvenido a bordo, marinero! ¿En qué puedo ayudarte hoy en esta travesía pirata?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Que sabes sobre Pikachu?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='¡Arrr, Eric! Pikachu es un Pokémon muy famoso, un ratoncito amarillo con una cola en forma de rayo y mejillas rojas que pueden generar electricidad. Es el Pokémon eléctrico por excelencia y es la mascota del mundo Pokémon. Fue creado para los juegos Pokémon Rojo y Azul y desde entonces ha aparecido en muchos juegos, cartas y hasta en la serie animada como compañero del valiente Ash Ketchum.\n\nPikachu puede evolucionar a Raichu usando una piedra trueno, y tiene un pre-evolución llamado Pichu. Es muy querido por su diseño adorable y es un ícono no solo de Pokémon,

In [91]:
agent_executor.invoke({"input": "Cual es la versión mas cara?"})



> Entering new AgentExecutor chain...

Invoking: `get-most-expensive-pokemon-card` with `{'pokemon_name': 'Pikachu'}`


Most Expensive Pikachu Card: Pikachu ★ - Market Price: $2066.66


> Finished chain.


{'input': 'Cual es la versión mas cara?',
 'chat_history': [HumanMessage(content='Hola! mi nombre es Eric!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='¡Ahoy, Eric! ¡Bienvenido a bordo, marinero! ¿En qué puedo ayudarte hoy en esta travesía pirata?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Que sabes sobre Pikachu?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='¡Arrr, Eric! Pikachu es un Pokémon muy famoso, un ratoncito amarillo con una cola en forma de rayo y mejillas rojas que pueden generar electricidad. Es el Pokémon eléctrico por excelencia y es la mascota del mundo Pokémon. Fue creado para los juegos Pokémon Rojo y Azul y desde entonces ha aparecido en muchos juegos, cartas y hasta en la serie animada como compañero del valiente Ash Ketchum.\n\nPikachu puede evolucionar a Raichu usando una piedra trueno, y tiene un pre-evolución llamado Pichu. Es muy querido por su diseño adorable y es un ícono no solo de Poké

In [92]:
agent_executor.invoke({"input": "¿Qué sabes sobre Charizard? ¿Cual es la versión más cara de carta?"})



> Entering new AgentExecutor chain...

Invoking: `search-wikipedia` with `{'query': 'Charizard'}`


Page: Charizard
Summary: Charizard (  CHAR-iz-ard), known in Japan as Lizardon (Japanese: リザードン, Hepburn: Rizādon), is a Pokémon in Nintendo and Game Freak's Pokémon franchise. Created by Atsuko Nishida, Charizard first appeared in the video games Pokémon Red and Blue (Pokémon Red and Green in Japan) and subsequent sequels. They have later appeared in various merchandise, spinoff titles and animated and printed adaptations of the franchise.  Shin-ichiro Miki voices Charizard in both the Japanese and English-language versions of the anime. An orange, dragon-like Pokémon, Charizard is the evolved form of Charmeleon and the final evolution of Charmander. It also has two "Mega Evolved" forms, Mega Charizard X and Y, that were likely both designed by Tomohiro Kitakaze, the designer of Mega Charizard X. It also has a Gigantamax form in Pokémon Sword and Shield, which changes its appearance a

{'input': '¿Qué sabes sobre Charizard? ¿Cual es la versión más cara de carta?',
 'chat_history': [HumanMessage(content='Hola! mi nombre es Eric!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='¡Ahoy, Eric! ¡Bienvenido a bordo, marinero! ¿En qué puedo ayudarte hoy en esta travesía pirata?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Que sabes sobre Pikachu?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='¡Arrr, Eric! Pikachu es un Pokémon muy famoso, un ratoncito amarillo con una cola en forma de rayo y mejillas rojas que pueden generar electricidad. Es el Pokémon eléctrico por excelencia y es la mascota del mundo Pokémon. Fue creado para los juegos Pokémon Rojo y Azul y desde entonces ha aparecido en muchos juegos, cartas y hasta en la serie animada como compañero del valiente Ash Ketchum.\n\nPikachu puede evolucionar a Raichu usando una piedra trueno, y tiene un pre-evolución llamado Pichu. Es muy querido por su diseño 

No ha sido capaz de realizar dos acciones... Podemos usar un agente ReAct:


En este caso necesitamos un Prompt para ejecutar ReAct. Podemos usar el de Langchain por ejemplo: [https://smith.langchain.com/hub/hwchase17/react-chat](https://smith.langchain.com/hub/hwchase17/react-chat)


In [100]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import OpenAI

prompt = hub.pull("hwchase17/react")

tools = [get_most_expensive_pokemon_card, search_wikipedia]

llm = OpenAI(api_key = api_key, model="gpt-4o-mini")

agent = create_react_agent(llm, tools, prompt)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [101]:
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               return_intermediate_steps=True
                               )

In [102]:
result = agent_executor.invoke({"input": "Es caro Charizard? De que tipo es?"})



> Entering new AgentExecutor chain...
 Necesito buscar información sobre Charizard para saber de qué tipo es.
Action: search-wikipedia
Action Input: CharizardPage: Charizard
Summary: Charizard (  CHAR-iz-ard), known in Japan as Lizardon (Japanese: リザードン, Hepburn: Rizādon), is a Pokémon in Nintendo and Game Freak's Pokémon franchise. Created by Atsuko Nishida, Charizard first appeared in the video games Pokémon Red and Blue (Pokémon Red and Green in Japan) and subsequent sequels. They have later appeared in various merchandise, spinoff titles and animated and printed adaptations of the franchise.  Shin-ichiro Miki voices Charizard in both the Japanese and English-language versions of the anime. An orange, dragon-like Pokémon, Charizard is the evolved form of Charmeleon and the final evolution of Charmander. It also has two "Mega Evolved" forms, Mega Charizard X and Y, that were likely both designed by Tomohiro Kitakaze, the designer of Mega Charizard X. It also has a Gigantamax form i

In [103]:
print(result)

{'input': 'Es caro Charizard? De que tipo es?', 'output': 'Most Expensive Charizard Card: Charizard ★ δ - Market Price: $993.99', 'intermediate_steps': [(AgentAction(tool='search-wikipedia', tool_input='Charizard', log=' Necesito buscar información sobre Charizard para saber de qué tipo es.\nAction: search-wikipedia\nAction Input: Charizard'), 'Page: Charizard\nSummary: Charizard (  CHAR-iz-ard), known in Japan as Lizardon (Japanese: リザードン, Hepburn: Rizādon), is a Pokémon in Nintendo and Game Freak\'s Pokémon franchise. Created by Atsuko Nishida, Charizard first appeared in the video games Pokémon Red and Blue (Pokémon Red and Green in Japan) and subsequent sequels. They have later appeared in various merchandise, spinoff titles and animated and printed adaptations of the franchise.  Shin-ichiro Miki voices Charizard in both the Japanese and English-language versions of the anime. An orange, dragon-like Pokémon, Charizard is the evolved form of Charmeleon and the final evolution of Cha